In [ ]:
"""
Using the JSON output of Fastp produces a table and plots for each sample
In: json fastp outputs
Out: all_poolseq_report.tsv table with summary of all samples
     png plots for quality and base content for each sample
"""
import glob
import json
import matplotlib.pyplot as plt

def plot_quality(quality_curves, filename, filter):   
   pos=list(range(1,151))
   plt.rcParams['font.size'] = 6
   plt.figure(figsize=(4, 2))
   plt.ylim(0,40)
   plt.ylabel("quality")
   plt.xlabel("position")   
   plt.title(filename)
   plt.plot(pos, quality_curves, linestyle="-", color="darkblue", lw=1)
   plt.style.use("ggplot")
   plt.tight_layout()
   plot_filename = "./02.qc_report/" + filter + "/" +filename.replace('.fq.gz','.qual.png')
   plt.savefig(plot_filename, dpi=300)
   plt.close()

def plot_base_content(content_curves, filename, filter):   
   pos=list(range(1,151))   
   plt.rcParams['font.size'] = 6
   plt.figure(figsize=(4, 2))
   plt.ylim(0,1)   
   plt.ylabel("base ratio")
   plt.title(filename)
   plt.xlabel("position")   
   plt.plot(pos, content_curves["A"], linestyle="-", color="darkblue", lw=1)
   plt.plot(pos, content_curves["T"], linestyle="-", color="darkred", lw=1)
   plt.plot(pos, content_curves["G"], linestyle="-", color="darkgreen", lw=1)
   plt.plot(pos, content_curves["C"], linestyle="-", color="orange", lw=1)
   plt.legend(["A", "T", "G", "C"], loc="upper right")
   plt.style.use("ggplot")   
   plt.tight_layout()
   plot_filename = "./02.qc_report/" + filter + "/" +filename.replace('.fq.gz','.base.png')
   plt.savefig(plot_filename, dpi=300)
   plt.close()

json_files = glob.glob("./02.qc_report/json_reports/*.json")

data = {}
ips_genome_size = 224977219
col_names = ["Idn     ", "Year", "Time", "Region", "Country", "Reps", \
             "After", "Gbp", "Cov", "Length", "GC",\
             "Insert", "Dups", "q20A", \
              "Before",  "Lost", "LQual", "Ns", "Short", "q20B", "Qubit", "%Reads_lost", "GCbefore", "File"]
header = "\t".join(col_names)

sample_qubit = {i.rstrip().split()[1] : i.split()[0] for i in open("./02.qc_report/sample_qubit") }


for file in json_files:
    with open(file, 'r') as f:
        fastp_out = json.load(f)        
        prefix = file.split("/")[3].replace(".qc_report.json", "")
        prefix_short = prefix[:12]
        idn = "_".join(prefix.split("_")[:3])        
               
        year = prefix.split("_")[2]
        time = prefix.split("_")[1][0]
        region = prefix.split("_")[0][0]
        country = prefix.split("_")[0][1:]
        replicate = prefix.split("_")[1][1]
        
        reads_after = fastp_out["summary"]["after_filtering"]["total_reads"]
        total_bp = fastp_out["summary"]["after_filtering"]["total_bases"]
        coverage = total_bp/ips_genome_size
        length = (fastp_out["summary"]["after_filtering"]["read1_mean_length"] + fastp_out["summary"]["after_filtering"]["read2_mean_length"])*0.5
        gc_content = fastp_out["summary"]["after_filtering"]["gc_content"]
        
        insert = fastp_out["insert_size"]["peak"]
        dups = fastp_out["duplication"]["rate"]
        over_q20 = fastp_out["summary"]["after_filtering"]["q20_rate"]
        before_q20 = fastp_out["summary"]["before_filtering"]["q20_rate"]

        reads_before = fastp_out["summary"]["before_filtering"]["total_reads"]
        reads_lost = reads_before - reads_after
        low_qual = fastp_out["filtering_result"]["low_quality_reads"]
        many_Ns = fastp_out["filtering_result"]["too_many_N_reads"]
        too_short = fastp_out["filtering_result"]["too_short_reads"]
        
        filename = file.split('/')[3].replace("qc_report.json", "fq.gz")
        quality_curves_after = fastp_out["read1_after_filtering"]["quality_curves"]["mean"]
        content_curves_after = fastp_out["read1_after_filtering"]["content_curves"]
        plot_quality(quality_curves_after, filename, "after")
        plot_base_content(content_curves_after, filename, "after")

        quality_curves_before = fastp_out["read1_before_filtering"]["quality_curves"]["mean"]
        content_curves_before = fastp_out["read1_before_filtering"]["content_curves"]
        plot_quality(quality_curves_before, filename, "before")
        plot_base_content(content_curves_before, filename, "before")

        qubit = sample_qubit[prefix_short]
        gc_content_before = fastp_out["summary"]["before_filtering"]["gc_content"]
        total_bp_before = fastp_out["summary"]["before_filtering"]["total_bases"]
        pct_reads_lost = round(100 - (reads_after/reads_before)*100, 1)
        
        row = [idn, year, time, country, region, replicate, \
               reads_after, total_bp, coverage, length, gc_content,\
               insert, dups, over_q20,\
               reads_before, reads_lost, low_qual, many_Ns, too_short, before_q20, qubit, pct_reads_lost, gc_content_before, filename]
               
        if idn not in data.keys():
            data[idn] = row            
        else:
            #sum:  "reads_after", "total_bp", "reads_before", "reads_lost", "low_qual", "many_Ns", "too_short"
            index_to_sum = [6, 7, 14, 15, 16, 17, 18]
            for k in index_to_sum:
                data[idn][k] += row[k]
            #Calculate coverage            
            coverage = data[idn][7]/ips_genome_size
            data[idn][8] = coverage            
            #Append filename
            data[idn][-1] += "," + row[-1]
            #Reads lost
            pct_reads_lost = round(100 - (data[idn][6]/data[idn][14])*100, 1)
            data[idn][21] = pct_reads_lost

data_sort = sorted(data.items(), key=lambda x: x[1][1:6])
data_sort_new=[]
out = open('./02.qc_report/all_poolseq_report.tsv', 'w')
out.write(header + '\n')
for k, v in data_sort:    
    after =  round(v[6]/1e6, 1)
    elems=v[:6]+ [round(v[6]/1e6, 1) ,round(v[7]/1e9, 1), round(v[8], 1), int(v[9]), round(v[10]*100,1), \
          int(v[11]), round(v[12]*100,1), round(v[13]*100,1), \
            round(v[14]/1e6, 1), round(v[15]/1e6, 1),round(v[16]/1e6, 1),round(v[17]/1e6, 1), round(v[18]/1e6, 1), \
                round(v[19]*100,1), v[20], v[21], round(v[22]*100, 1), v[23]]
    elems_str = list(map(str, elems))
    data_sort_new.append(elems)
    out.write("\t".join(elems_str) + "\n")

out.close()


In [ ]:
"""
Generates an html file with a table summarizing the Fastp report, one data sample per row
"""

html = """<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>Ips poolseq report</title>
    <style>
        tr:nth-child(odd) {
            background-color: #ffffff; /* Light gray */
        }
        tr:nth-child(even) {
            background-color: #f2f2f2; /* White */
        }
        table { border-collapse: collapse; width: 100%; }
        th, td { border: 1px solid #ccc;  text-align: center; vertical-align: middle; padding: 0 8px; }
        th { background-color: #ddd; padding: 6px; border-color:#bbb}
        img { max-width: 250px; height: auto; }
        body {font-family: sans-serif; font-size:12px}
        .top_header { background-color: #d3d3d3; padding: 6px; border-color:#bbb}
        .reads{font-family:mono; font-size:10px; text-align:left}
        .red_samples {
        color: red;
        font-weight: bold;
    }
    </style>  
  <script>
  window.onload = function() {    
    const table = document.getElementById('Tab');  
      for (const row of table.tBodies[0].rows) {
      const cov = row.cells[9];
      const val = parseFloat(cov.textContent);

        const gc = row.cells[11];
        const val_gc = parseFloat(gc.textContent);

        if (!isNaN(val_gc)) {
          if (val_gc > 40) {
            gc.style.backgroundColor = '#FFD9E6';
        } else if(val_gc > 50){
        gc.style.backgroundColor = '#F4A6A6';
        }}

        const gcb = row.cells[21];
        const val_gcb = parseFloat(gcb.textContent);

        if (!isNaN(val_gcb)) {
          if (val_gcb > 40 && val_gcb <50) {
            gcb.style.backgroundColor = '#FFD9E6';
        } else if(val_gcb > 50){
        gcb.style.backgroundColor = '#F4A6A6';
        }}


      if (!isNaN(val)) {
        if (val < 100) {
          cov.style.backgroundColor = '#ff9999';
        } else if (val < 200) {
          cov.style.backgroundColor = '#ffcc99';
        } else if (val < 300) {
          cov.style.backgroundColor = '#ffffcc';
        } else {
          cov.style.backgroundColor = '#99ff99';  // val >= 300
        }
      }
    }
  };


</script>
</head>
<body>    
<br>
    <table id="Tab">
    <thead>
        <tr>
        <th colspan=6; class="top_header">Dataset</th>
        <th colspan=8; class="top_header">Reads after filtering</th>
        <th colspan=15; class="top_header">Reads before filtering</th>
        </tr>
        <tr>
        <th>Prefix</th>
        <th>Year</th>
        <th>Time</th>
        <th>Country</th>
        <th>Region</th>
        <th>Rep</th>
        <th>Qubit</th>
        
        <th>Reads after (M)</th>
        <th>Gbp total</th>
        <th>Coverage</th>
        <th>Read length</th>
        <th>GC (%)</th>
        <th>Q20 after</th>
        <th>Quality plot after QC</th>
        <th>Base ratio plot after QC</th>
        
        <th>Reads before (M)</th>
        <th>Reads discarded (%)</th>
        <th>Reads discarded (M)</th>
        <th>Low qual (M)</th>
        <th>Many Ns (M)</th>
        <th>Short reads (M)</th>
        <th>GC before(%)</th>
        <th>Q20 before</th>     
        <th>Insert bp</th>
        <th>Duplics (%)</th>   
        <th>Quality plot before QC</th>
        <th>Base ratio plot before QC</th>

        <th>Files</th>       
        <th>Overrepresented reads (in 1e6 sample) </th>   
        </tr> 
        </thead>
  <tbody>"""

import subprocess

for row in data_sort_new:        
    idn, year, time, country, region, rep, after, gbp, cov, length, gc, insert , \
      dups, q20a, before, lost, lqual, ns, short, q20b, qubit, pct_lost, gc_before, file = row
    
    top5_reads_out = subprocess.run(f"grep -A12 {idn} ./02.qc_report/top5_reads | egrep -v '^0|_' ", shell=True, capture_output=True, text=True)
    top5_reads = top5_reads_out.stdout.replace("1.fq", "").replace("2.fq", "")
    
    plotQ = "./before/" + file.replace(".fq.gz",".qual.png").split(',')[0]
    plotB = "./before/" + file.replace(".fq.gz",".base.png").split(',')[0]
    plotQa = "./after/" + file.replace(".fq.gz",".qual.png").split(',')[0]
    plotBa = "./after/" + file.replace(".fq.gz",".base.png").split(',')[0]
    html += f"""
        <tr>
            <td>{idn}</td>
            <td>{year}</td>
            <td>{time}</td>
            <td>{country}</td>
            <td>{region}</td>
            <td>{rep}</td>
            <td>{qubit}</td>

            <td>{after}</td>
            <td>{gbp}</td>
            <td>{cov}</td>
            <td>{length}</td>
            <td>{gc}</td>
            <td>{q20a}</td>
            <td><img src="{plotQa}" /></td>
            <td><img src="{plotBa}" /></td>
                                             
            <td>{before}</td>
            <td>{pct_lost}</td>
            <td>{lost}</td>
            <td>{lqual}</td>
            <td>{ns}</td>
            <td>{short}</td>
            <td>{gc_before}</td>
            <td>{q20b}</td>
            <td>{insert}</td>
            <td>{dups}</td>
            <td><img src="{plotQ}" /></td>
            <td><img src="{plotB}" /></td>

            <td>{file.replace(',','<br>')}</td>
            <td class="reads"><pre>{top5_reads}</pre></td>
            
        </tr>
    """

html += """
   </tbody> </table>
<p id="command">
    fastp -i ${files[0]} 
        -I ${files[1]} 
        -o ./ips_reads/$prefix/$out1
        -O ./ips_reads/$prefix/$out2 
        -w 16 
        --detect_adapter_for_pe 
        --trim_poly_g 
        --length_required 50 
        --qualified_quality_phred 20 
        -j ./ips_reads/$prefix/$json 
        -h ./ips_reads/$prefix/$html
        <br>
</p>
   </body>

   <script>
    const keywords = ["SFIN_Ea_2015", "SFIN_Eb_2015", "WFIN_Ea_2015", "WFIN_Eb_2015", "SFIN_La_2016", "SFIN_Lb_2016r", "SFIN_Ea_2017", "SFIN_Eb_2017", "SFIN_La_2017", "SFIN_Lb_2017"]; // Your keywords

    document.querySelectorAll("td").forEach(cell => {
        keywords.forEach(keyword => {
            if (cell.textContent.includes(keyword)) {
                cell.classList.add("red_samples");
            }
        });
    });
</script>
</html>
"""

# Write to file
with open("./02.qc_report/ips_poolseq_report.html", "w", encoding="utf-8") as f:
    f.write(html)
